In [6]:
import pandas as pd

nodes_df = pd.read_csv('C:\data\data_nodes.csv')
edges_df = pd.read_csv('C:\data\data_edges.csv')

nodes_df.head(),edges_df.head()

(  conservation_id          location   data_type access_level  \
 0            C017            Serang    Personal   Restricted   
 1            C001         Aceh Jaya  Geographic       Public   
 2            C011         Pontianak    Personal   Restricted   
 3            C007  Minahasa Selatan  Geographic       Public   
 4            C002           Takalar    Personal   Restricted   
 
    transaction_count  
 0                  1  
 1                  2  
 2                  1  
 3                  1  
 4                  1  ,
   source target  weight
 0   C014   C003       1
 1   C001   C005       1
 2   C005   C001       1
 3   C004   C015       1
 4   C015   C004       1)

In [7]:
from pyvis.network import Network
import networkx as nx
from datetime import datetime

# Inisialisasi jaringan
net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True)

# Konfigurasi global
net.barnes_hut(gravity=-80000, central_gravity=0.3, spring_length=250, spring_strength=0.001, damping=0.09, overlap=0)

# Tambahkan node
for idx, row in nodes_df.iterrows():
  node_size = 10 + row['transaction_count'] * 2
  node_title = f"ID: {row['conservation_id']}<br>Lokasi: {row['location']}<br>Tipe: {row['data_type']}<br>Akses: {row['access_level']}<br>Transaksi: {row['transaction_count']}"
  
  # Tentukan warna berdasarkan tipe data
  if row['data_type'] == 'Geographic':
    color = '#4CAF50' # Hijau
  elif row['data_type'] == 'Personal':
    color = '#2196F3' # Biru
  else:
    color = '#FF5722' # Oranye
  
  # Tentukan bentuk berdasarkan level akses
  shape = 'dot' if row['access_level'] == 'Public' else 'diamond' if row['access_level'] == 'Restricted' else 'star'
  
  net.add_node(
    n_id=row['conservation_id'],
    label=row['conservation_id'],
    title=node_title,
    size=node_size,
    color=color,
    shape=shape,
    borderWidth=2
  )

# Tambahkan edge
for idx, row in edges_df.iterrows():
  net.add_edge(
    row['source'],
    row['target'],
    value=row['weight'],
    title=f"{row['weight']} transaksi",
    color='#757575'
  )

# Konfigurasi tampilan
net.set_options("""
{
"nodes": {
"font": {
"size": 12,
"face": "arial"
}
},
"edges": {
"smooth": {
"type": "continuous"
}
},
"physics": {
"forceAtlas2Based": {
"gravitationalConstant": -50,
"centralGravity": 0.01,
"springLength": 100,
"springConstant": 0.08
},
"minVelocity": 0.75,
"solver": "forceAtlas2Based"
}
}
""")

# Simpan dan tampilkan
net.show("blockchain_network.html")
net.show("blockchain_network.html")

blockchain_network.html
blockchain_network.html


In [9]:
# Analisis centrality menggunakan NetworkX
G = nx.Graph()

# Tambahkan node dan edge
for idx, row in nodes_df.iterrows():
  G.add_node(row['conservation_id'], node_type=row['data_type'], access=row['access_level'])

for idx, row in edges_df.iterrows():
  G.add_edge(row['source'], row['target'], weight=row['weight'])

# Hitung metrik centrality
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
closeness_centrality = nx.closeness_centrality(G)

# Gabungkan hasil
centrality_df = pd.DataFrame({
  'conservation_id': list(degree_centrality.keys()),
  'Degree': list(degree_centrality.values()),
  'Betweenness': list(betweenness_centrality.values()),
  'Closeness': list(closeness_centrality.values())
})

# Gabungkan dengan data asli
centrality_df = centrality_df.merge(nodes_df, on='conservation_id')

# Tampilkan 5 node paling penting
centrality_df.sort_values('Betweenness', ascending=False).head(5)

,conservation_id,Degree,Betweenness,Closeness,location,data_type,access_level,transaction_count
0,C017,0.000000,0.0,0.000000,Serang,Personal,Restricted,1
1,C001,0.052632,0.0,0.052632,Aceh Jaya,Geographic,Public,2
2,C011,0.000000,0.0,0.000000,Pontianak,Personal,Restricted,1
3,C007,0.000000,0.0,0.000000,Minahasa Selatan,Geographic,Public,1
4,C002,0.052632,0.0,0.052632,Takalar,Personal,Restricted,1
